In [1]:
import os

import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms 
from sklearn import metrics, model_selection, preprocessing
import torch
from torchvision import datasets
from torch.utils.data import DataLoader, Dataset 

from Model import *
from callbacks import EarlyStopping
from efficientnet_pytorch import EfficientNet
from utils import *

INPUT_PATH = "images/"
TRAIN_PATH = "images/train/"
VALID_PATH = "images/valid/"
MODEL_PATH = "models/"
MODEL_NAME = "EfficientNet_model_Changed_"
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 20
IMAGE_SIZE = 256

In [2]:
class LeafModel(Model):
    def __init__(self, num_classes, init=nn.init.kaiming_normal_):
        super().__init__()

        self.effnet = EfficientNet.from_pretrained("efficientnet-b3")
        self.head = create_head(1536, num_classes)

    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}

    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt

    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        outputs = self.head(x)

        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [3]:
train_aug = transforms.Compose([transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation((-20, 20)),
                                transforms.RandomAffine(degrees=5, translate=(0.10, 0.15)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                transforms.RandomErasing(p=0.4)
                                ])
valid_aug = transforms.Compose([transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

In [4]:
train_dataset = datasets.ImageFolder("images/train/", transform=train_aug)
valid_dataset = datasets.ImageFolder("images/valid/", transform=valid_aug)

In [5]:
train_dataset.classes

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper_bell___Bacterial_spot',
 'Pepper_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

In [6]:
 torch.cuda.current_device()

0

In [7]:
torch.cuda.get_device_name(0)

'GeForce RTX 3070'

In [8]:
model = LeafModel(num_classes=len(train_dataset.classes))

Loaded pretrained weights for efficientnet-b3


In [10]:
es = EarlyStopping(
        monitor="valid_loss",
        model_path=os.path.join(MODEL_PATH, MODEL_NAME + "_early.bin"),
        patience=3,
        mode="min")

In [11]:
model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=TRAIN_BATCH_SIZE,
        valid_bs=VALID_BATCH_SIZE,
        device="cuda",
        epochs=EPOCHS,
        fp16=True,
        callbacks=[es])

100%|███████████████████████████████████████| 476/476 [00:35<00:00, 13.45it/s, accuracy=0.98, loss=0.0649, stage=Valid]


Validation score improved (inf --> 0.06492789004647528). Saving model!


100%|██████████████████████████████████████| 476/476 [00:36<00:00, 12.89it/s, accuracy=0.987, loss=0.0428, stage=Valid]


Validation score improved (0.06492789004647528 --> 0.04278222206799519). Saving model!


100%|███████████████████████████████████████| 476/476 [00:36<00:00, 13.03it/s, accuracy=0.99, loss=0.0333, stage=Valid]


...

EarlyStopping counter: 1 out of 3


  0%|                                                                                         | 0/1905 [00:00<?, ?it/s]

EarlyStopping counter: 2 out of 3


100%|█████████████████████████████████████| 476/476 [00:36<00:00, 13.21it/s, accuracy=0.998, loss=0.00798, stage=Valid]

EarlyStopping counter: 3 out of 3


In [9]:
model.load('models/EfficientNet_model_Changed__early.bin')

In [10]:
model.save_model(os.path.join(MODEL_PATH, MODEL_NAME + "Final_onlyModel.pth"))

In [10]:
%debug

> d:\gautham\plantdisease\callbacks\early_stopping.py(30)on_epoch_end()
     28 
     29     def on_epoch_end(self, model):
---> 30         epoch_score = model.metrics[self.model_state][self.monitor_value]
     31         if self.mode == "min":
     32             score = -1.0 * epoch_score



ipdb>  self.monitor_value


'loss'


ipdb>  model.metrics


{'train': {'accuracy': 0.8697086936731907, 'loss': 0.47133112254610937}, 'valid': {}, 'test': {}}


ipdb>  model.metrics['valid']['loss']


*** KeyError: 'loss'


ipdb>  model.metrics['train']['loss']


0.47133112254610937


ipdb>  n


In [18]:
train_aug = transforms.Compose([transforms.Resize((IMAGE_SIZE,IMAGE_SIZE)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.RandomRotation((-20, 20)),
                                transforms.RandomAffine(degrees=5, translate=(0.10, 0.15)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                transforms.RandomErasing(p=0.4)
                                ])
valid_aug = transforms.Compose([transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

In [19]:
train_dataset = datasets.ImageFolder("images/train/", transform=train_aug)
valid_dataset = datasets.ImageFolder("images/valid/", transform=valid_aug)

In [20]:
model.load('models/EfficientNet_model_new_early.bin')

In [21]:
es = EarlyStopping(
        monitor="valid_loss",
        model_path=os.path.join(MODEL_PATH, MODEL_NAME + "_early2.bin"),
        patience=3,
        mode="min")

In [22]:
model.fit(
        train_dataset,
        valid_dataset=valid_dataset,
        train_bs=TRAIN_BATCH_SIZE,
        valid_bs=VALID_BATCH_SIZE,
        device="cuda",
        epochs=EPOCHS,
        callbacks=[es])

100%|██████████████████████████████████████| 1113/1113 [00:50<00:00, 21.86it/s, accuracy=0.96, loss=0.123, stage=Valid]


Validation score improved (inf --> 0.12253258741287673). Saving model!


100%|██████████████████████████████████████| 1113/1113 [00:51<00:00, 21.70it/s, accuracy=0.96, loss=0.121, stage=Valid]


Validation score improved (0.12253258741287673 --> 0.1213094911780491). Saving model!


100%|█████████████████████████████████████| 1113/1113 [00:52<00:00, 21.17it/s, accuracy=0.963, loss=0.116, stage=Valid]


...

EarlyStopping counter: 1 out of 3


  0%|                                                                                         | 0/4413 [00:00<?, ?it/s]

EarlyStopping counter: 2 out of 3


100%|█████████████████████████████████████| 1113/1113 [00:52<00:00, 21.28it/s, accuracy=0.967, loss=0.107, stage=Valid]

EarlyStopping counter: 3 out of 3


In [12]:
model.save_model(os.path.join(MODEL_PATH, MODEL_NAME + "2_onlyModel.pth"))

In [5]:
model.load('models/EfficientNet_model_early1.bin')

In [14]:
model.save_model(os.path.join(MODEL_PATH, MODEL_NAME + "Final_onlyModel.pth"))

In [6]:
from sklearn.metrics import classification_report, confusion_matrix

In [7]:
test_aug = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])
test_dataset = datasets.ImageFolder("images/test/", transform=test_aug)

In [40]:
y_pred_gen = model.predict(valid_dataset, device='cuda')

In [21]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 33
    Root location: images/test/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [60]:
valid_dataset.

Dataset ImageFolder
    Number of datapoints: 17572
    Root location: images/valid/
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [ ]:
class TestDataset(Dataset):
    def __init__(self, path):
        self.path = path
    
    def __len__(self):
        return len(path)
    
    def __getitem__(self, idx):
        return sample

In [41]:
y_pred = []
for data in y_pred_gen:
    val = np.argmax(data, axis=1)
    y_pred.append(list(val))

100%|████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 16.38it/s, stage=Test]


In [42]:
def flatten(input):
    new_list = []
    for i in input:
        for j in i:
            new_list.append(j)
    return new_list

In [46]:
y_pred = np.array(flatten(y_pred))

In [48]:
y_label = []
for i in y_pred:
    y_label.append(train_dataset.classes[i])

In [49]:
y_label

['Apple___Cedar_apple_rust',
 'Apple___Cedar_apple_rust',
 'Apple___Cedar_apple_rust',
 'Apple___Cedar_apple_rust',
 'Apple___Apple_scab',
 'Apple___Apple_scab',
 'Apple___Apple_scab',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Common_rust_',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Potato___Early_blight',
 'Potato___healthy',
 'Potato___healthy',
 'Tomato___Early_blight',
 'Tomato___Early_blight',
 'Tomato___Early_blight',
 'Tomato___Early_blight',
 'Tomato___Early_blight',
 'Tomato___Early_blight',
 'Tomato___healthy',
 'Tomato___healthy',
 'Tomato___healthy',
 'Tomato___healthy',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [200]:
def show_batch(img,labels, batch=6):
    fig, ax = plt.subplots(3,3, figsize=(12,10))
    k = 0
    for i in range(3):
        for j in range(3):
            ax[i][j].imshow(img[k].permute(1, 2, 0))
            ax[i][j].title.set_text(train_data.classes[labels[k]])
            ax[i][j].xaxis.set_visible(False)
            ax[i][j].yaxis.set_visible(False)
            k+=1
    fig.tight_layout()

In [ ]:
show_img(img, label)
plt.savefig('batch.png', bbox_inches = 'tight');